In [6]:
import av
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Функция логарифмического поиска для оценки движения
def logarithmic_search(current_block, ref_frame, block_size, search_radius):
    h, w = ref_frame.shape
    best_mse = float('inf')
    best_mv = (0, 0)
    for step in [4, 2, 1]:  # Steps of the logarithmic search
        for y in range(-step * search_radius, step * search_radius + 1, step):
            for x in range(-step * search_radius, step * search_radius + 1, step):
                ref_y, ref_x = y, x
                if 0 <= ref_y <= h - block_size and 0 <= ref_x <= w - block_size:
                    ref_block = ref_frame[ref_y:ref_y+block_size, ref_x:ref_x+block_size]
                    mse = np.mean((current_block - ref_block) ** 2)
                    if mse < best_mse:
                        best_mse = mse
                        best_mv = (ref_y, ref_x)
    return best_mv


# Функция для обработки видео
def process_video(input_file, Q):
    container = av.open(input_file)
    original_frames = []
    reconstructed_frames = []
    ref_frame = None
    
    for frame in container.decode(video=0):
        y_component = np.array(frame.to_ndarray(format='gray'))
        h, w = y_component.shape
        if ref_frame is None:
            ref_frame = np.zeros((h, w), dtype=np.uint8)
        residual_frames = np.zeros_like(y_component, dtype=np.float32)
        
        for i in range(0, h, 16):
            for j in range(0, w, 16):
                current_block = y_component[i:i+16, j:j+16]
                if i+16 <= h and j+16 <= w:
                    mv = logarithmic_search(current_block, ref_frame, 16, 8)
                    ref_y, ref_x = i + mv[0], j + mv[1]
                    if 0 <= ref_y <= h - 16 and 0 <= ref_x <= w - 16:
                        ref_block = ref_frame[ref_y:ref_y+16, ref_x:ref_x+16]
                        residual = current_block - ref_block
                        residual_frames[i:i+16, j:j+16] = residual
        
        # Применяем DCT, квантование и обратную DCT для восстановления
        dct_residual = cv2.dct(residual_frames)
        quantized_residual = np.round(dct_residual / Q)
        dequantized_residual = quantized_residual * Q
        idct_residual = cv2.idct(dequantized_residual)
        reconstructed_frame = idct_residual + ref_frame
        
        # Сохраняем кадры для вычисления PSNR и степени сжатия
        original_frames.append(y_component)
        reconstructed_frames.append(np.clip(reconstructed_frame, 0, 255).astype(np.uint8))
        ref_frame = reconstructed_frame  # Обновляем опорный кадр
    
    return original_frames, reconstructed_frames


# Функция для вычисления PSNR
def calculate_psnr(original_frame, reconstructed_frame):
    mse = np.mean((original_frame - reconstructed_frame) ** 2)
    if mse == 0:
        return 100  # В случае идентичных кадров
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Функция для вычисления степени сжатия
def calculate_compression_ratio(original_frame, reconstructed_frame):
    original_size = original_frame.size * original_frame.itemsize
    compressed_size = np.count_nonzero(reconstructed_frame) * reconstructed_frame.itemsize
    compression_ratio = original_size / compressed_size
    return compression_ratio

def main():
    Q_values = [10, 20, 30, 40, 50]
    psnr_values = []
    compression_ratios = []

    for Q in Q_values:
        original_frames, reconstructed_frames = process_video('lr1_1.avi', Q)
        psnr = calculate_psnr(original_frames[-1], reconstructed_frames[-1])
        compression_ratio = calculate_compression_ratio(original_frames[-1], reconstructed_frames[-1])
        psnr_values.append(psnr)
        compression_ratios.append(compression_ratio)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(Q_values, psnr_values)
    plt.xlabel('Quantization Step Q')
    plt.ylabel('PSNR')
    plt.title('PSNR vs Quantization Step Q')

    plt.subplot(1, 2, 2)
    plt.plot(Q_values, compression_ratios)
    plt.xlabel('Quantization Step Q')
    plt.ylabel('Compression Ratio')
    plt.title('Compression Ratio vs Quantization Step Q')
    plt.show()

if __name__ == "__main__":
    main()



KeyboardInterrupt: 